In [2]:
import numpy as np
import pandas as pd

In [3]:
users = pd.read_csv("data/Users.csv")
useractivity = pd.read_csv("data/UserActivity.csv")
compsp = pd.read_csv("data/CompetitionPartipation.csv")
comps = pd.read_csv("data/Competition.csv")
blogs = pd.read_csv("data/Blogs.csv")
comments = pd.read_csv("data/Comments.csv")
discussion = pd.read_csv("data/Discussion.csv")
jobs = pd.read_csv("data/Jobs.csv")

In [114]:
original_months = {2:7, 3:8, 4:9, 5:10, 6:11, 7:12, 8:1, 9:2, 10:3, 11:4, 12:5, 1:6}
users['Created At Month'] = users['Created At Month'].map(original_months).astype('int')
useractivity['datetime Month'] = useractivity['datetime Month'].map(original_months).astype('int')
compsp['Created At Month'] = compsp['Created At Month'].map(original_months).astype('int')
blogs['Published At Month'] = blogs['Published At Month'].map(original_months).astype('int')
comments['Created At Month'] = comments['Created At Month'].map(original_months).astype('int')
discussion['Created At Month'] = discussion['Created At Month'].map(original_months).astype('int')

# print(useractivity['datetime Month'].unique())
# print(users['Created At Month'].unique())
# print(comps[''])
# sys.exit()

users.loc[(users['Created At Month'] >= 1) & (users['Created At Month'] <= 5),'Created At Year'] = 2
useractivity.loc[(useractivity['datetime Month'] >= 1) & (useractivity['datetime Month'] <= 5),'datetime Year'] = 2
compsp.loc[(compsp['Created At Month'] >= 1) & (compsp['Created At Month'] <= 5),'Created At Year'] = 2
blogs.loc[(blogs['Published At Month'] >= 1) & (blogs['Published At Month'] <= 5),'Published At Year'] = 2
comments.loc[(comments['Created At Month'] >= 1) & (comments['Created At Month'] <= 5),'Created At Year'] = 2
discussion.loc[(discussion['Created At Month'] >= 1) & (discussion['Created At Month'] <= 5),'Created At Year'] = 2

users['creation_date'] = users.apply(lambda x : pd.Timestamp(year=x['Created At Year'],month=x['Created At Month'],day=x['Created At Day_of_month']),axis=1)
useractivity['activity_date'] = useractivity.apply(lambda x : pd.Timestamp(year=x['datetime Year'],month=x['datetime Month'],day=x['datetime Day_of_month']),axis=1)
compsp['participation_date'] = compsp.apply(lambda x : pd.Timestamp(year=x['Created At Year'],month=x['Created At Month'],day=x['Created At Day_of_month']),axis=1)
blogs['Publish_date'] = blogs.apply(lambda x : pd.Timestamp(year=x['Published At Year'],month=x['Published At Month'],day=x['Published At Day_of_month']),axis=1)
comments['comment_date'] = comments.apply(lambda x : pd.Timestamp(year=x['Created At Year'],month=x['Created At Month'],day=x['Created At Day_of_month']),axis=1)
discussion['discussion_date'] = discussion.apply(lambda x : pd.Timestamp(year=x['Created At Year'],month=x['Created At Month'],day=x['Created At Day_of_month']),axis=1)


In [115]:
useractivitymerged = useractivity.merge(users[['User_ID','creation_date']],on='User_ID',how='left')
mask = (useractivitymerged["activity_date"] >= useractivitymerged["creation_date"]) & (useractivitymerged["activity_date"] < useractivitymerged["creation_date"] + pd.DateOffset(months=1))

In [116]:
mask_activities = (~(useractivitymerged['Title'].str.startswith('job'))) \
        & (~(useractivitymerged['Title'].str.startswith('comp')))\
        & (~(useractivitymerged['Title'].str.startswith('blog')))\
        & (~(useractivitymerged['Title'].str.startswith('Signed Up')))\
        & (~(useractivitymerged['Title'].str.startswith('$create_alias')))\
        & (~(useractivitymerged['Title'].str.startswith('$identify')))\
        & (~(useractivitymerged['Title'].str.startswith('badge')))
keys = (useractivitymerged['Title'][mask_activities]).unique()
rest_activities = useractivitymerged[useractivitymerged["Title"].isin(keys) & (mask)]
counts = rest_activities.groupby(["User_ID", "Title"]).size().unstack(fill_value=0)
users = users.merge(counts, on="User_ID", how="left")
users[keys] = users[keys].fillna(0)

In [118]:
users.describe()

,FeatureX,FeatureY,Created At Year,Created At Month,Created At Day_of_month,Accepted Team Leadership Transfer,Applied To Job,Changed Password,Confirmed Email,Created Submission,...,Updated Profile,Updated Submission,Updated Team,Viewed All Competitions,Viewed All Discussions,Viewed All Jobs,Viewed All Learning Pages,Viewed Discussion,Viewed FAQ,Votes (Up/Down)
count,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,...,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000
mean,0.085314,0.528237,1.261339,7.122855,16.305970,0.000725,0.007814,0.000322,0.698461,0.675582,...,1.075566,0.154435,0.002900,3.485539,2.974623,0.280110,1.556836,0.062112,0.006203,0.007734
std,0.279359,0.891619,0.439382,1.928793,9.023629,0.026918,0.159115,0.017949,0.580066,4.722159,...,2.770278,1.487215,0.073961,7.135913,6.131134,1.395606,2.196281,0.534132,0.103329,0.161381
min,0.000000,0.000000,1.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,5.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,7.000000,17.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,2.000000,9.000000,25.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,0.000000,3.000000,3.000000,0.000000,2.000000,0.000000,0.000000,0.000000
max,1.000000,3.000000,2.000000,10.000000,31.000000,1.000000,7.000000,1.000000,19.000000,136.000000,...,55.000000,78.000000,4.000000,182.000000,289.000000,40.000000,74.000000,24.000000,4.000000,7.000000


In [105]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12413 entries, 0 to 12412
Data columns (total 40 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   FeatureX                              12413 non-null  int64  
 1   FeatureY                              12413 non-null  int64  
 2   User_ID                               12413 non-null  object 
 3   Countries_ID                          6531 non-null   object 
 4   Created At time                       12413 non-null  object 
 5   Created At Year                       12413 non-null  int64  
 6   Created At Month                      12413 non-null  int64  
 7   Created At Day_of_month               12413 non-null  int64  
 8   creation_date                         12413 non-null  object 
 9   Accepted Team Leadership Transfer     10303 non-null  float64
 10  Applied To Job                        10303 non-null  float64
 11  Changed Passwor

In [74]:
def get_per_user(row):
    account_creation_date = row['creation_date']
    activity_per_user = useractivity.loc[(row['User_ID'] == useractivity['User_ID']) 
                                       & (useractivity['activity_date'] >= account_creation_date) 
                                       & (useractivity['activity_date'] < account_creation_date + pd.DateOffset(months=1))
                                       & (mask)]
    counts = activity_per_user['Title'].value_counts()
    counts_list = [int(counts.get(key,0)) for key in keys]
    return pd.Series(counts_list)


In [75]:
users[keys] = users.apply(get_per_user,axis=1)

KeyboardInterrupt: 

In [57]:
pd.Series(useractivity.loc[(useractivity['User_ID'] == "ID_U1IJB9D9") & (mask)]['Title'].value_counts())

Title
Created Submission           82
Viewed All Competitions      39
Viewed All Discussions        8
Viewed All Learning Pages     3
Updated Discussion            1
Confirmed Email               1
Joined Competition            1
Name: count, dtype: int64

In [44]:
compsp[compsp['User_ID'] == 'ID_U1IJB9D9']

,Competition ID,Participant Type,Successful Submission Count,User_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month
6918,ID_528W,1,count 3,ID_U1IJB9D9,20:31:17.006706,1,1,16


In [45]:
useractivity.loc[useractivity['User_ID'] == 'ID_U1IJB9D9']['Title'].value_counts()

Title
Created Submission           82
Viewed All Competitions      39
comp_ID_528W                 10
Viewed All Discussions        8
Viewed All Learning Pages     3
comp_ID_540G                  1
Updated Discussion            1
$create_alias                 1
$identify                     1
Signed Up                     1
badge_OCZE                    1
Confirmed Email               1
Joined Competition            1
Name: count, dtype: int64

In [41]:
comps[comps['Comp_ID'] == 'ID_IV5D']

,FeatureA,FeatureB,FeatureC,FeatureD,FeatureE,SecretCode,FeatureF,FeatureG,FeatureH,FeatureI,Country_ID,Comp_ID,End Time time,End Time Year,End Time Month,End Time Day_of_month,Start Time time,Start Time Year,Start Time Month,Start Time Day_of_month
163,[5],[],10.0,0,2,1,[2],[],[2],10.0,DYYF,ID_IV5D,14:00:00,1.0,11.0,23.0,20:00:00,1,11,22


In [38]:
useractivity['Title'].unique()

array(['comp_ID_IV5D', 'comp_ID_M3ST', 'comp_ID_ZN92', 'comp_ID_DD5S',
       'comp_ID_1261', 'comp_ID_X0PD', 'comp_ID_L3EB', 'comp_ID_JPR9',
       'comp_ID_TDAE', 'comp_ID_Q8XJ', 'comp_ID_PDKH', 'comp_ID_NKJR',
       'comp_ID_AL6C', 'comp_ID_JX23', 'comp_ID_YDM3', 'comp_ID_8B8I',
       'comp_ID_UU4Y', 'comp_ID_56O8', 'comp_ID_GR92', 'comp_ID_I9BN',
       'comp_ID_M3G0', 'comp_ID_V0FX', 'comp_ID_528W', 'comp_ID_JRC2',
       'comp_ID_F7QD', 'comp_ID_D3SA', 'comp_ID_0SN1', 'comp_ID_GWL0',
       'comp_ID_WOVD', 'comp_ID_VGMZ', 'comp_ID_J883', 'comp_ID_Y6FW',
       'comp_ID_6E1C', 'comp_ID_PB3Q', 'comp_ID_3RMM', 'comp_ID_P8BQ',
       'comp_ID_3URQ', 'comp_ID_8LGP', 'comp_ID_RBCT', 'comp_ID_Y4Y2',
       'comp_ID_CFN7', 'comp_ID_1DWD', 'comp_ID_GVGW', 'comp_ID_KESL',
       'comp_ID_20XL', 'comp_ID_W18I', 'comp_ID_U2NJ', 'comp_ID_PJZC',
       'comp_ID_MN35', 'comp_ID_IRWB', 'comp_ID_XRP7', 'comp_ID_HCST',
       'comp_ID_1CTF', 'comp_ID_AS79', 'comp_ID_CNBR', 'comp_ID_J5IZ',
      

In [3]:
original_months = {2:7, 3:8, 4:9, 5:10, 6:11, 7:12, 8:1, 9:2, 10:3, 11:4, 12:5, 1:6}

In [4]:
users['Created At Month'] = users['Created At Month'].map(original_months)

In [ ]:
pd.to_datetime()

In [16]:
users['Creation_date'] = users.apply(lambda x : pd.Timestamp(f"{x['Created At Month']}-{x['Created At Day_of_month']}"),axis=1) 

In [17]:
users

,FeatureX,FeatureY,User_ID,Countries_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month,Creation_date
0,0,0,ID_DC6S4E9O,X9GR,19:33:13.663391,1,6,16,0001-06-16 00:00:00
1,1,0,ID_E8S97OUT,X9GR,22:18:18.228921,1,5,27,0001-05-27 00:00:00
2,0,0,ID_QZ1HASL3,X9GR,23:13:03.266635,1,6,1,0001-06-01 00:00:00
3,1,0,ID_SDV9Q3E9,X9GR,09:50:09.697573,1,5,27,0001-05-27 00:00:00
4,1,0,ID_W942LN7F,X9GR,13:33:06.531662,1,6,16,0001-06-16 00:00:00
...,...,...,...,...,...,...,...,...,...
12408,0,1,ID_C5393PD0,7DKX,05:02:39.352427,1,4,6,0001-04-06 00:00:00
12409,0,1,ID_QX7XRT0C,K4DG,02:50:19.027320,1,8,25,0001-08-25 00:00:00
12410,0,1,ID_RGN2RK66,ONON,22:06:33.713692,1,5,7,0001-05-07 00:00:00
12411,0,0,ID_XZS1QE0W,3YQR,06:56:02.570173,1,10,27,0001-10-27 00:00:00


In [ ]:
submission_count_keys = compsp['Successful Submission Count'].unique()
submission_count_keys = submission_count_keys[~pd.isna(submission_count_keys)]
def get_comp_per_user(row):
    account_creation_day = int(row['Created At Day_of_month'])
    account_creation_month = int(row['Created At Month'])
    nextmonth = (account_creation_month + 1) % 12
    nextday = (account_creation_day + 1) % 31
    competitions_per_user = compsp.loc[(row['User_ID'] == compsp['User_ID']) 
                                       & (((compsp['Created At Month'] == account_creation_month) 
                                           & (compsp['Created At Day_of_month'] >= account_creation_day)) 
                                           |((compsp['Created At Month'] == nextmonth) 
                                           & (compsp['Created At Day_of_month'] < nextday)))]
    submission_counts = competitions_per_user['Successful Submission Count'].value_counts()
    submission_counts = [int(submission_counts.get(key,0)) for key in submission_count_keys]
    return pd.Series([len(competitions_per_user)]+submission_counts)

def get_comments_per_user(row):
    account_creation_month = row['Created At Month']
    comments_per_user = comments.loc[(row['User_ID'] == comments['User_ID']) & (comments['Created At Month'] == account_creation_month) ]
    return len(comments_per_user)

def get_discussion_per_user(row):
    account_creation_month = row['Created At Month']
    discussions_per_user = discussion.loc[(row['User_ID'] == discussion['User_ID']) & (discussion['Created At Month'] == account_creation_month) ]
    return len(discussions_per_user)

def split_time(row):
    time = pd.Timestamp(row['Created At time'])
    hour = int(time.hour)
    minutes = int(time.minute)
    seconds = int(time.second)
    return pd.Series([hour,minutes,seconds])


In [289]:
jobs_activity = useractivity[useractivity['Title'].str.startswith('job')]
unique_jobs = jobs_activity.drop_duplicates(subset=['Title','User_ID'])
job_count = unique_jobs.groupby('User_ID')['Title'].nunique()
job_count = pd.Series(job_count)
users['job_count'] = users['User_ID'].map(job_count).fillna(0).astype('int')

In [290]:
blogs_activity = useractivity[useractivity['Title'].str.startswith('job')]
unique_blogs = blogs_activity.drop_duplicates(subset=['Title','User_ID'])
blog_count = unique_blogs.groupby('User_ID')['Title'].nunique()
blog_count = pd.Series(blog_count)
users['blog_activity_count'] = users['User_ID'].map(blog_count).fillna(0).astype('int')

In [291]:
submission_count_keys = list(map(lambda x : "subm "+ x,submission_count_keys))
users[['competitons_count'] + submission_count_keys] = users.apply(get_comp_per_user,axis=1)
users['comments_count'] = users.apply(get_comments_per_user,axis=1)
users['discussions_count'] = users.apply(get_discussion_per_user,axis=1)
users[['hour','minute','second']] = users.apply(split_time,axis=1)

In [351]:
useractivitymerged = useractivity.merge(users[['User_ID','Created At Month','Created At Day_of_month']],on='User_ID',how='left')

useractivitymergedfiltered = useractivitymerged.loc[((useractivitymerged['datetime Month'] == useractivitymerged['Created At Month'])
                                                    & (useractivitymerged['datetime Day_of_month'] >= useractivitymerged['Created At Day_of_month'])) 
                                                    | ((useractivitymerged['datetime Month'] == (useractivitymerged['Created At Month'] + 1)%12)
                                                    & (useractivitymerged['datetime Day_of_month'] < (useractivitymerged['Created At Day_of_month']+1)%31))]

users['activity_days_count'] = users['User_ID'].map(useractivitymergedfiltered['User_ID'].value_counts()).fillna(0).astype('int')

In [352]:
useractivitymergedfilteredtarget = useractivitymerged.loc[((useractivitymerged['datetime Month'] == (useractivitymerged['Created At Month']+1)%12)
                                                    & (useractivitymerged['datetime Day_of_month'] >= (useractivitymerged['Created At Day_of_month']+1)%31)) 
                                                    | ((useractivitymerged['datetime Month'] == (useractivitymerged['Created At Month'] + 2)%12)
                                                    & (useractivitymerged['datetime Day_of_month'] < (useractivitymerged['Created At Day_of_month']+2)%31))]


In [335]:
users

,FeatureX,FeatureY,User_ID,Countries_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month,job_count,blog_activity_count,...,subm count 6,subm count 7,subm count 5,subm count 4,comments_count,discussions_count,hour,minute,second,activity_days_count
0,0,0,ID_DC6S4E9O,X9GR,19:33:13.663391,1,1,16,0,0,...,0,0,0,0,0,0,19,33,13,11
1,1,0,ID_E8S97OUT,X9GR,22:18:18.228921,1,12,27,2,2,...,0,0,0,0,0,0,22,18,18,184
2,0,0,ID_QZ1HASL3,X9GR,23:13:03.266635,1,1,1,0,0,...,0,0,0,0,0,0,23,13,3,21
3,1,0,ID_SDV9Q3E9,X9GR,09:50:09.697573,1,12,27,0,0,...,0,0,0,0,0,0,9,50,9,105
4,1,0,ID_W942LN7F,X9GR,13:33:06.531662,1,1,16,0,0,...,0,0,0,0,2,0,13,33,6,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12408,0,1,ID_C5393PD0,7DKX,05:02:39.352427,1,11,6,0,0,...,0,0,0,0,0,0,5,2,39,0
12409,0,1,ID_QX7XRT0C,K4DG,02:50:19.027320,1,3,25,0,0,...,0,0,0,0,0,0,2,50,19,12
12410,0,1,ID_RGN2RK66,ONON,22:06:33.713692,1,12,7,0,0,...,0,0,0,0,0,0,22,6,33,26
12411,0,0,ID_XZS1QE0W,3YQR,06:56:02.570173,1,5,27,0,0,...,0,0,0,0,0,0,6,56,2,9


In [353]:
useractivitymergedfilteredtarget = users['User_ID'].map(useractivitymergedfilteredtarget['User_ID'].value_counts() > 0).fillna(0).astype('int')

In [366]:
useractivitymergedfilteredtarget.sum()

np.int64(1800)

In [338]:
discussion

,Competition ID,Personal,Theme,Disc_ID,User_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month
0,GeneralDiscussion,1,NaN,ID_0BK7TH,ID_AAGFVLHH,19:48:26.862175,1,12,27
1,GeneralDiscussion,1,NaN,ID_ZFJ9J7,ID_0AJGQILU,09:37:52.039526,1,12,20
2,GeneralDiscussion,1,NaN,ID_ME45XX,ID_U90KK6KL,17:50:33.894800,1,2,9
3,ID_45W9,1,NaN,ID_2BG9IX,ID_U90KK6KL,07:57:55.178845,1,2,10
4,GeneralDiscussion,1,NaN,ID_251XVV,ID_F7HIGFHQ,19:56:13.041893,1,12,27
...,...,...,...,...,...,...,...,...,...
1434,GeneralDiscussion,1,NaN,ID_09W0TZ,ID_AAJKI0P3,14:15:31.081484,1,5,23
1435,GeneralDiscussion,1,NaN,ID_613AUJ,ID_8A3X5IWU,11:06:11.201934,1,5,14
1436,GeneralDiscussion,1,NaN,ID_TCN8TB,ID_3NL0QLE6,04:57:31.578153,1,5,12
1437,GeneralDiscussion,1,NaN,ID_FTJ37Z,ID_3PL1LQQX,05:11:59.313759,1,5,9


In [339]:
discussionmerged = discussion.merge(users[['User_ID','Created At Month','Created At Day_of_month']],on='User_ID',how='left')


In [340]:
discussionmerged

,Competition ID,Personal,Theme,Disc_ID,User_ID,Created At time,Created At Year,Created At Month_x,Created At Day_of_month_x,Created At Month_y,Created At Day_of_month_y
0,GeneralDiscussion,1,NaN,ID_0BK7TH,ID_AAGFVLHH,19:48:26.862175,1,12,27,12,20
1,GeneralDiscussion,1,NaN,ID_ZFJ9J7,ID_0AJGQILU,09:37:52.039526,1,12,20,12,20
2,GeneralDiscussion,1,NaN,ID_ME45XX,ID_U90KK6KL,17:50:33.894800,1,2,9,2,8
3,ID_45W9,1,NaN,ID_2BG9IX,ID_U90KK6KL,07:57:55.178845,1,2,10,2,8
4,GeneralDiscussion,1,NaN,ID_251XVV,ID_F7HIGFHQ,19:56:13.041893,1,12,27,12,27
...,...,...,...,...,...,...,...,...,...,...,...
1434,GeneralDiscussion,1,NaN,ID_09W0TZ,ID_AAJKI0P3,14:15:31.081484,1,5,23,5,16
1435,GeneralDiscussion,1,NaN,ID_613AUJ,ID_8A3X5IWU,11:06:11.201934,1,5,14,5,11
1436,GeneralDiscussion,1,NaN,ID_TCN8TB,ID_3NL0QLE6,04:57:31.578153,1,5,12,5,6
1437,GeneralDiscussion,1,NaN,ID_FTJ37Z,ID_3PL1LQQX,05:11:59.313759,1,5,9,5,9


In [345]:
discussionmergedfilteredtarget = discussionmerged.loc[((discussionmerged['Created At Month_x'] == (discussionmerged['Created At Month_y']+1)%12)
                                                    & (discussionmerged['Created At Day_of_month_x'] >= (discussionmerged['Created At Day_of_month_y']+1)%31)) 
                                                    | ((discussionmerged['Created At Month_x'] == (discussionmerged['Created At Month_y'] + 2)%12)
                                                    & (discussionmerged['Created At Day_of_month_x'] < (discussionmerged['Created At Day_of_month_y']+2)%31))]


In [346]:
discussionmergedfilteredtarget['User_ID'].value_counts() > 0

User_ID
ID_CEDNKSVV    True
ID_6FWILK4H    True
ID_0HI8E3M8    True
ID_AQIBV924    True
ID_PNADKFXR    True
               ... 
ID_0W7CRJ5S    True
ID_WZ6WNU2T    True
ID_EPAX9S0O    True
ID_5FD3I6FB    True
ID_EKFB430Z    True
Name: count, Length: 92, dtype: bool

In [347]:
discussionmergedfilteredtarget = users['User_ID'].map(discussionmergedfilteredtarget['User_ID'].value_counts() > 0).fillna(0).astype('int')

In [363]:
discussionmergedfilteredtarget.sum()

np.int64(92)

In [354]:
commentsmerged = comments.merge(users[['User_ID','Created At Month','Created At Day_of_month']],on='User_ID',how='left')
commentsmergedfilteredtarget = commentsmerged.loc[((commentsmerged['Created At Month_x'] == (commentsmerged['Created At Month_y']+1)%12)
                                                    & (commentsmerged['Created At Day_of_month_x'] >= (commentsmerged['Created At Day_of_month_y']+1)%31)) 
                                                    | ((commentsmerged['Created At Month_x'] == (commentsmerged['Created At Month_y'] + 2)%12)
                                                    & (commentsmerged['Created At Day_of_month_x'] < (commentsmerged['Created At Day_of_month_y']+2)%31))]
commentsmergedfilteredtarget = users['User_ID'].map(commentsmergedfilteredtarget['User_ID'].value_counts() > 0).fillna(0).astype('int')

In [364]:
commentsmergedfilteredtarget.sum()

np.int64(36)

In [360]:
target = (commentsmergedfilteredtarget) | (discussionmergedfilteredtarget) | (useractivitymergedfilteredtarget)

In [361]:
target.sum()

np.int64(1815)

In [367]:
users['target'] = target

In [369]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12413 entries, 0 to 12412
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   FeatureX                 12413 non-null  int64 
 1   FeatureY                 12413 non-null  int64 
 2   User_ID                  12413 non-null  object
 3   Countries_ID             6531 non-null   object
 4   Created At time          12413 non-null  object
 5   Created At Year          12413 non-null  int64 
 6   Created At Month         12413 non-null  int64 
 7   Created At Day_of_month  12413 non-null  int64 
 8   job_count                12413 non-null  int64 
 9   blog_activity_count      12413 non-null  int64 
 10  competitons_count        12413 non-null  int64 
 11  subm count 10            12413 non-null  int64 
 12  subm count 3             12413 non-null  int64 
 13  subm count 8             12413 non-null  int64 
 14  subm count 9             12413 non-nul

In [119]:
data = pd.read_csv('data/data.csv')

In [124]:
(data['target'] == 0).sum() / (data['target'] == 1).sum()

np.float64(20.038983050847456)

In [125]:
sample = pd.read_csv("data/SampleSubmission.csv")

In [129]:
sample['User_ID_Next_month_Activity'].str.replace("_Month_5","")

0       ID_4TOXNBGB
1       ID_CHFTIP26
2       ID_FU5GMWLQ
3       ID_254TVBQP
4       ID_4ENO2VFE
           ...     
1335    ID_PI8N2KJX
1336    ID_L9AHTUD7
1337    ID_BB7GLV87
1338    ID_WB6O1NFS
1339    ID_99LD9C1B
Name: User_ID_Next_month_Activity, Length: 1340, dtype: object

In [130]:
useractivity['User_ID']

0         ID_RT43AK77
1         ID_RT43AK77
2         ID_RT43AK77
3         ID_RT43AK77
4         ID_RT43AK77
             ...     
317287    ID_Z759BCS3
317288    ID_WNJMADF8
317289    ID_7NBY9MV2
317290    ID_LHPJJSOK
317291    ID_LCFNGARD
Name: User_ID, Length: 317292, dtype: object

In [138]:
useractivity.loc[useractivity['User_ID'] == "ID_ZZMHNLNT"]

,User_ID,Title,datetime time,datetime Year,datetime Month,datetime Day_of_month
280825,ID_ZZMHNLNT,Signed Up,11:55:11,1,4,1
280826,ID_ZZMHNLNT,$create_alias,11:55:12,1,4,1
280827,ID_ZZMHNLNT,$identify,11:55:12,1,4,1
280828,ID_ZZMHNLNT,Viewed All Competitions,11:55:15,1,4,1
280829,ID_ZZMHNLNT,Viewed All Learning Pages,11:55:15,1,4,1
280830,ID_ZZMHNLNT,Viewed All Discussions,11:55:16,1,4,1
280831,ID_ZZMHNLNT,badge_OCZE,11:57:10,1,4,1
280832,ID_ZZMHNLNT,Confirmed Email,11:57:10,1,4,1


In [137]:
users[users['User_ID'] == 'ID_ZZMHNLNT']

,FeatureX,FeatureY,User_ID,Countries_ID,Created At time,Created At Year,Created At Month,Created At Day_of_month
2017,0,0,ID_ZZMHNLNT,NaN,10:55:11.435972,1,4,1


In [140]:
users['Created At Month'].unique()

array([ 1, 12,  2, 11,  3,  5,  4])

In [141]:
useractivity['datetime Month'].unique()

array([11, 12,  5,  4,  1,  3,  2])

In [151]:
datatrain = pd.read_csv("data/datatrain.csv")

In [153]:
datatrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9068 entries, 0 to 9067
Data columns (total 57 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   FeatureX                              9068 non-null   int64  
 1   FeatureY                              9068 non-null   int64  
 2   User_ID                               9068 non-null   object 
 3   Countries_ID                          9068 non-null   int64  
 4   Created At time                       9068 non-null   object 
 5   Created At Year                       9068 non-null   int64  
 6   Created At Month                      9068 non-null   int64  
 7   Created At Day_of_month               9068 non-null   int64  
 8   job_activity_count                    9068 non-null   int64  
 9   comp_activity_count                   9068 non-null   int64  
 10  blog_activity_count                   9068 non-null   int64  
 11  badge_activity_co

In [3]:
datatrain = pd.read_csv("data/datatrain.csv")

In [8]:
datatrain.dtypes

FeatureX                                  int64
FeatureY                                  int64
User_ID                                  object
Countries_ID                              int64
Created At time                          object
Created At Year                           int64
Created At Month                          int64
Created At Day_of_month                   int64
job_activity_count                        int64
comp_activity_count                       int64
blog_activity_count                       int64
badge_activity_count                      int64
Accepted Team Leadership Transfer       float64
Applied To Job                          float64
Changed Password                        float64
Confirmed Email                         float64
Created Submission                      float64
Created Team                            float64
Deleted Team                            float64
Downloaded Competition Datafile         float64
Invited Member To Team                  

In [23]:
(datatrain.loc[(datatrain.drop(columns=["User_ID","Countries_ID","Created At time","Created At Month","Created At Year","Created At Day_of_month","hour","minute","second","target"]).sum(axis=1) > 200)]).target

1649    1
5018    1
5205    1
5372    1
6290    1
7987    1
8083    1
8518    1
8679    1
8841    0
Name: target, dtype: int64